<a href="https://colab.research.google.com/github/Ruin9999/Ruin9999/blob/main/comfyui_mount.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
from pathlib import Path

current_dir = !pwd
WORKSPACE = f"{current_dir[0]}/ComfyUI"

!echo "Mounting Google Drive..."
%cd /
drive.mount('/content/drive')

WORKSPACE = "/content/drive/MyDrive/ComfyUI"
%cd /content/drive/MyDrive

![ ! -d $WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $WORKSPACE

!echo -= Updating ComfyUI =-
!git pull

!echo -= Install dependencies =-
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu124
!pip3 install -r requirements.txt

%cd custom_nodes
![ ! -d ComfyUI-Manager ] && echo -= Initial setup ComfyUI-Manager =- && git clone https://github.com/ltdrdata/ComfyUI-Manager.git
%cd ComfyUI-Manager
!git pull
!pip3 install -r requirements.txt

%cd $WORKSPACE

!echo -= Install custom nodes dependencies =-
![ -f "custom_nodes/ComfyUI-Manager/scripts/colab-dependencies.py" ] && python "custom_nodes/ComfyUI-Manager/scripts/colab-dependencies.py"

print("\nAll done\n")

Mounting Google Drive...
/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive
/content/drive/MyDrive/ComfyUI
-= Updating ComfyUI =-
Already up to date.
-= Install dependencies =-
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu124
/content/drive/MyDrive/ComfyUI/custom_nodes
-= Initial setup ComfyUI-Manager =-
Cloning into 'ComfyUI-Manager'...
remote: Enumerating objects: 13982, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 13982 (delta 4), reused 9 (delta 3), pack-reused 13965 (from 1)
Receiving objects: 100% (13982/13982), 13.79 MiB | 17.35 MiB/s, done.
Resolving deltas: 100% (10288/10288), done.
/content/drive/MyDrive/ComfyUI/custom_nodes/ComfyUI-Manager
Already up to date.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### Run ComfyUI with cloudflared (Recommended Way)




In [ ]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    #print(l, end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server

--2024-10-05 11:17:14--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2024.9.1/cloudflared-linux-amd64.deb [following]
--2024-10-05 11:17:14--  https://github.com/cloudflare/cloudflared/releases/download/2024.9.1/cloudflared-linux-amd64.deb
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/106867604/5ff19c3d-e567-440e-9529-4bd2c1237b83?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=releaseassetproduction%2F20241005%2Fus-east-1%2Fs3%2Faws4_request&X-Amz-Date=20241005T111714Z&X-Amz-Expires=300&X-Amz-Signature=4d03afa2a049f749c7e8258c1488ce7064efb3be809738cf86b3fc768456